## Data Loading: Forecast Output

In [66]:
import pandas as pd
import numpy as np
forecast_df=pd.read_csv("Forecast.csv")

In [89]:
forecast_df

,part_id,part_type,location_id,region,forecast_date,forecasted_demand,forecast_lower_bound,forecast_upper_bound,unit_cost,revenue,method_used,demand_pattern,replenishment_strategy,ABC_Class,XYZ_Class,volume_class,volatility_class,mae,rmse,mase,rmsse,bias,avg_cost_impact,base_lead_time,adjusted_lead_time,lead_time_mean,lead_time_std_dev,lead_time_min,lead_time_max,lead_time_p90,lead_time_p95,forecast_date_str,part_prefix,customer_id,service_level,z_value,ordering_cost_per_order,holding_cost_rate,stockout_cost_per_unit,holding_cost_per_unit,demand_variance_during_lt,safety_stock_units,expected_demand_during_lt,reorder_point_units,annual_demand_units,cv_demand,intermittent_adjustment,eoq_units,cycle_stock_units,avg_inventory_level_units,abc_xyz_safety_multiplier,safety_stock_units_adjusted,avg_inventory_level_units_adjusted,expected_shortage_per_cycle,fill_rate,days_of_supply,achieved_service_level
0,BRAKE_PAD_1_005,BRAKE_PAD_TYPE_1,TX_3,TX,2011-01-29,2.000000,1.600000,2.400000,2.94,5.88,TSB,Lumpy,Project-Based,A,Z,High Volume,Unknown,0.7314,1.1619,0.790,0.7338,0.0086,6.37,3.09,3.55,3.55,0.89,2.13,7.10,4.62,5.33,20110129,BRA,CUST_TXTX3BP1_LUM_AZ_20110129_BRA_0,0.98,2.053749,800,0.15,14.70,0.4410,4.792541,4.496041,7.100000,11.596041,730.000000,0.580950,1.290475,2100.156806,1050.078403,1054.574444,1.3,5.844853,1055.923257,0.105996,0.985071,527.287222,0.95
1,BRAKE_PAD_1_005,BRAKE_PAD_TYPE_1,TX_3,TX,2011-01-30,2.100000,1.680000,2.520000,2.94,8.82,TSB,Lumpy,Project-Based,A,Z,High Volume,Highly Stable,0.7314,1.1619,0.790,0.7338,0.0086,6.37,3.09,3.54,3.54,0.89,2.13,7.09,4.61,5.31,20110130,BRA,CUST_TXTX3BP1_LUM_AZ_20110130_BRA_1,0.98,2.053749,800,0.15,14.70,0.4410,4.779041,4.489704,7.434000,11.923704,766.500000,0.553286,1.276643,2128.953598,1064.476799,1068.966503,1.3,5.836615,1070.313415,0.105847,0.985762,509.031668,0.95
2,BRAKE_PAD_1_005,BRAKE_PAD_TYPE_1,TX_3,TX,2011-01-31,2.190000,1.752000,2.628000,2.94,8.82,TSB,Lumpy,Project-Based,A,Z,High Volume,Highly Stable,0.7314,1.1619,0.790,0.7338,0.0086,6.37,3.09,3.33,3.33,0.83,2.00,6.66,4.33,4.99,20110131,BRA,CUST_TXTX3BP1_LUM_AZ_20110131_BRA_2,0.98,2.053749,800,0.15,14.70,0.4410,4.495539,4.354499,7.292700,11.647199,799.350000,0.530548,1.265274,2154.734477,1077.367239,1081.721738,1.3,5.660849,1083.028087,0.102659,0.985923,493.936867,0.95
3,BRAKE_PAD_1_005,BRAKE_PAD_TYPE_1,TX_3,TX,2011-02-01,2.171000,1.736800,2.605200,2.94,5.88,TSB,Lumpy,Project-Based,A,Z,High Volume,Highly Stable,0.7314,1.1619,0.790,0.7338,0.0086,6.37,2.61,2.87,2.87,0.72,1.72,5.74,3.73,4.30,20110201,BRA,CUST_TXTX3BP1_LUM_AZ_20110201_BRA_3,0.98,2.053749,800,0.15,14.70,0.4410,3.874533,4.042565,6.230770,10.273335,792.415000,0.535191,1.267596,2149.303550,1074.651775,1078.694340,1.3,5.255335,1079.907110,0.095305,0.984704,496.865196,0.95
4,BRAKE_PAD_1_005,BRAKE_PAD_TYPE_1,TX_3,TX,2011-02-02,1.953900,1.563120,2.344680,2.94,0.00,TSB,Lumpy,Project-Based,A,Z,High Volume,Highly Stable,0.7314,1.1619,0.790,0.7338,0.0086,6.37,2.61,2.73,2.73,0.68,1.64,5.46,3.55,4.10,20110202,BRA,CUST_TXTX3BP1_LUM_AZ_20110202_BRA_4,0.98,2.053749,800,0.15,14.70,0.4410,3.685532,3.942733,5.334147,9.276880,713.173500,0.594657,1.297328,2086.835554,1043.417777,1047.360511,1.3,5.125554,1048.543331,0.092952,0.982574,536.035882,0.95
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
439867,LED_PANEL_2_149,LED_PANEL_TYPE_2,WI_2,WI,2016-04-20,0.240835,0.192668,0.289002,0.97,0.00,TSB,Lumpy,Project-Based,C,Z,Low Volume,Highly Volatile,0.3144,0.6536,0.929,0.7114,0.0059,0.97,6.22,7.16,7.16,1.79,4.29,14.32,9.31,10.74,20160420,LED,CUST_WIWI2LP2_LUM_CZ_20160420_LED_439867,0.90,1.281552,400,0.25,1.94,0.2425,1.661163,1.651742,1.724379,3.376121,87.904778,2.000000,2.000000,1077.023946,538.511973,540.163715,1.0,1.651742,540.163715,0.226193,0.868826,2242.878712,0.95
439868,LED_PANEL_2_149,LED_PANEL_TYPE_

##  Geocoding Distributor Locations

This script fetches geographic coordinates for internal location IDs using OpenStreetMap's Nominatim geocoder via `geopy`.

### 🔹 Purpose
Enrich internal facility codes (e.g., `TX_1`) with city names and their latitude/longitude for:
- Network design
- Route optimization
- Simulation models (e.g., AnyLogic)

### 🔹 Process
1. Map internal IDs to city names (`location_map`)
2. Geocode each city using `geopy.Nominatim`
3. Store results with a 1-second delay to respect API limits
4. Output as `coordinates_df` with:
   - `location_id`
   - `Assumed City`
   - `Latitude`, `Longitude`




In [ ]:
import pandas as pd
from geopy.geocoders import Nominatim
import time

# --- Create the mapping from your internal ID to a real city ---
location_map = {
    'TX_1': 'Dallas, TX',
    'TX_2': 'Houston, TX',
    'TX_3': 'Austin, TX',
    'CA_1': 'Los Angeles, CA',
    'CA_2': 'San Diego, CA',
    'CA_3': 'San Jose, CA',
    'CA_4': 'Sacramento, CA',
    'WI_1': 'Milwaukee, WI',
    'WI_2': 'Madison, WI',
    'WI_3': 'Green Bay, WI'
}


location_ids = ['TX_1', 'CA_4', 'WI_3', 'WI_2', 'CA_3', 'TX_2', 'TX_3', 'WI_1', 'CA_1', 'CA_2']


geolocator = Nominatim(user_agent="anylogistix_geocoder_app")

results = []
print("Fetching coordinates for each location...")

for loc_id in location_ids:
    city_name = location_map.get(loc_id)
    if city_name:
        try:
            # Get the location data
            location_data = geolocator.geocode(city_name)
            if location_data:
                results.append({
                    'location_id': loc_id,
                    'Assumed City': city_name,
                    'Latitude': round(location_data.latitude, 4),
                    'Longitude': round(location_data.longitude, 4)
                })
                print(f"  Successfully found coordinates for {loc_id} ({city_name})")
            else:
                print(f"  Could not find coordinates for {loc_id} ({city_name})")
            
           
            time.sleep(1) 

        except Exception as e:
            print(f"An error occurred while processing {loc_id}: {e}")
            results.append({
                'location_id': loc_id,
                'Assumed City': city_name,
                'Latitude': 0.0,
                'Longitude': 0.0
            })

# Convert the results to a pandas DataFrame for easy viewing
coordinates_df = pd.DataFrame(results)
coordinates_df

Fetching coordinates for each location...
  Successfully found coordinates for TX_1 (Dallas, TX)
  Successfully found coordinates for CA_4 (Sacramento, CA)
  Successfully found coordinates for WI_3 (Green Bay, WI)
  Successfully found coordinates for WI_2 (Madison, WI)
  Successfully found coordinates for CA_3 (San Jose, CA)
  Successfully found coordinates for TX_2 (Houston, TX)
  Successfully found coordinates for TX_3 (Austin, TX)
  Successfully found coordinates for WI_1 (Milwaukee, WI)
  Successfully found coordinates for CA_1 (Los Angeles, CA)
  Successfully found coordinates for CA_2 (San Diego, CA)


,location_id,Assumed City,Latitude,Longitude
0,TX_1,"Dallas, TX",32.7763,-96.7969
1,CA_4,"Sacramento, CA",38.5811,-121.4939
2,WI_3,"Green Bay, WI",44.5126,-88.0126
3,WI_2,"Madison, WI",43.0748,-89.3838
4,CA_3,"San Jose, CA",37.3362,-121.8906
5,TX_2,"Houston, TX",29.7589,-95.3677
6,TX_3,"Austin, TX",30.2711,-97.7437
7,WI_1,"Milwaukee, WI",43.0386,-87.9091
8,CA_1,"Los Angeles, CA",34.0537,-118.2428
9,CA_2,"San Diego, CA",32.7174,-117.1628


In [90]:
forecast_df.rename(columns={
    'location_id': 'Location'
}, inplace=True)

### Define location-to-coordinates mapping

In [91]:
# Define location-to-coordinates mapping
location_coordinates = pd.DataFrame({
    'Location': ['TX_1', 'CA_4', 'WI_3', 'WI_2', 'CA_3', 'TX_2', 'TX_3', 'WI_1', 'CA_1', 'CA_2'],
    'City': ['Dallas, TX', 'Sacramento, CA', 'Green Bay, WI', 'Madison, WI', 'San Jose, CA',
             'Houston, TX', 'Austin, TX', 'Milwaukee, WI', 'Los Angeles, CA', 'San Diego, CA'],
    'Latitude': [32.7763, 38.5811, 44.5126, 43.0748, 37.3362, 29.7589, 30.2711, 43.0386, 34.0537, 32.7174],
    'Longitude': [-96.7969, -121.4939, -88.0126, -89.3838, -121.8906, -95.3677,
                  -97.7437, -87.9091, -118.2428, -117.1628]
})


### Merge `forecast_df` with `location_coordinates` for further analyses

In [92]:
forecast_df=forecast_df.merge(location_coordinates,on="Location",how="inner")
forecast_df

,part_id,part_type,Location,region,forecast_date,forecasted_demand,forecast_lower_bound,forecast_upper_bound,unit_cost,revenue,method_used,demand_pattern,replenishment_strategy,ABC_Class,XYZ_Class,volume_class,volatility_class,mae,rmse,mase,rmsse,bias,avg_cost_impact,base_lead_time,adjusted_lead_time,lead_time_mean,lead_time_std_dev,lead_time_min,lead_time_max,lead_time_p90,lead_time_p95,forecast_date_str,part_prefix,customer_id,service_level,z_value,ordering_cost_per_order,holding_cost_rate,stockout_cost_per_unit,holding_cost_per_unit,demand_variance_during_lt,safety_stock_units,expected_demand_during_lt,reorder_point_units,annual_demand_units,cv_demand,intermittent_adjustment,eoq_units,cycle_stock_units,avg_inventory_level_units,abc_xyz_safety_multiplier,safety_stock_units_adjusted,avg_inventory_level_units_adjusted,expected_shortage_per_cycle,fill_rate,days_of_supply,achieved_service_level,City,Latitude,Longitude
0,BRAKE_PAD_1_005,BRAKE_PAD_TYPE_1,TX_3,TX,2011-01-29,2.000000,1.600000,2.400000,2.94,5.88,TSB,Lumpy,Project-Based,A,Z,High Volume,Unknown,0.7314,1.1619,0.790,0.7338,0.0086,6.37,3.09,3.55,3.55,0.89,2.13,7.10,4.62,5.33,20110129,BRA,CUST_TXTX3BP1_LUM_AZ_20110129_BRA_0,0.98,2.053749,800,0.15,14.70,0.4410,4.792541,4.496041,7.100000,11.596041,730.000000,0.580950,1.290475,2100.156806,1050.078403,1054.574444,1.3,5.844853,1055.923257,0.105996,0.985071,527.287222,0.95,"Austin, TX",30.2711,-97.7437
1,BRAKE_PAD_1_005,BRAKE_PAD_TYPE_1,TX_3,TX,2011-01-30,2.100000,1.680000,2.520000,2.94,8.82,TSB,Lumpy,Project-Based,A,Z,High Volume,Highly Stable,0.7314,1.1619,0.790,0.7338,0.0086,6.37,3.09,3.54,3.54,0.89,2.13,7.09,4.61,5.31,20110130,BRA,CUST_TXTX3BP1_LUM_AZ_20110130_BRA_1,0.98,2.053749,800,0.15,14.70,0.4410,4.779041,4.489704,7.434000,11.923704,766.500000,0.553286,1.276643,2128.953598,1064.476799,1068.966503,1.3,5.836615,1070.313415,0.105847,0.985762,509.031668,0.95,"Austin, TX",30.2711,-97.7437
2,BRAKE_PAD_1_005,BRAKE_PAD_TYPE_1,TX_3,TX,2011-01-31,2.190000,1.752000,2.628000,2.94,8.82,TSB,Lumpy,Project-Based,A,Z,High Volume,Highly Stable,0.7314,1.1619,0.790,0.7338,0.0086,6.37,3.09,3.33,3.33,0.83,2.00,6.66,4.33,4.99,20110131,BRA,CUST_TXTX3BP1_LUM_AZ_20110131_BRA_2,0.98,2.053749,800,0.15,14.70,0.4410,4.495539,4.354499,7.292700,11.647199,799.350000,0.530548,1.265274,2154.734477,1077.367239,1081.721738,1.3,5.660849,1083.028087,0.102659,0.985923,493.936867,0.95,"Austin, TX",30.2711,-97.7437
3,BRAKE_PAD_1_005,BRAKE_PAD_TYPE_1,TX_3,TX,2011-02-01,2.171000,1.736800,2.605200,2.94,5.88,TSB,Lumpy,Project-Based,A,Z,High Volume,Highly Stable,0.7314,1.1619,0.790,0.7338,0.0086,6.37,2.61,2.87,2.87,0.72,1.72,5.74,3.73,4.30,20110201,BRA,CUST_TXTX3BP1_LUM_AZ_20110201_BRA_3,0.98,2.053749,800,0.15,14.70,0.4410,3.874533,4.042565,6.230770,10.273335,792.415000,0.535191,1.267596,2149.303550,1074.651775,1078.694340,1.3,5.255335,1079.907110,0.095305,0.984704,496.865196,0.95,"Austin, TX",30.2711,-97.7437
4,BRAKE_PAD_1_005,BRAKE_PAD_TYPE_1,TX_3,TX,2011-02-02,1.953900,1.563120,2.344680,2.94,0.00,TSB,Lumpy,Project-Based,A,Z,High Volume,Highly Stable,0.7314,1.1619,0.790,0.7338,0.0086,6.37,2.61,2.73,2.73,0.68,1.64,5.46,3.55,4.10,20110202,BRA,CUST_TXTX3BP1_LUM_AZ_20110202_BRA_4,0.98,2.053749,800,0.15,14.70,0.4410,3.685532,3.942733,5.334147,9.276880,713.173500,0.594657,1.297328,2086.835554,1043.417777,1047.360511,1.3,5.125554,1048.543331,0.092952,0.982574,536.035882,0.95,"Austin, TX",30.2711,-97.7437
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
439867,LED_PANEL_2_149,LED_PANEL_TYPE_2,WI_2,WI,2016-04-20,0.240835,0.192668,0.289002,0.97,0.00,TSB,Lumpy,Project-Based,C,Z,Low Volume,Highly Volatile,0.3144,0.6536,0.929,0.7114,0.0059,0.97,6.22,7.16,7.16,1.79,4.29,14.32,9.31,10.74,20160420,LED,CUST_WIWI2LP2_LUM_CZ_20160420_LED_439867,0.90,1.281552,400,0.25,1.94,0.2425,1.661163,1.6

##  Forecast Feature Preprocessing Pipeline

This script prepares a clean and standardized feature matrix for machine learning or clustering using scikit-learn.

###  Purpose
To preprocess a supply chain forecast dataset by:
- Scaling numerical features
- Encoding categorical features

###  Features Used

**Numerical**
- `forecasted_demand`, `unit_cost`, `revenue`
- `lead_time_mean`, `lead_time_std_dev`, `cv_demand`
- `eoq_units`, `safety_stock_units`, `reorder_point_units`

**Categorical**
- `demand_pattern`, `ABC_Class`, `XYZ_Class`, `volatility_class`

###  Pipeline Steps
1. **Standardize** numerical values using `StandardScaler`
2. **Encode** categorical values with `OneHotEncoder` (ignores unseen labels)
3. **Transform** the combined features using `ColumnTransformer`
4. **Output**: `X_processed` – ready for ML models like KMeans, PCA, or classifiers


In [67]:
import pandas as pd
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer

# Copy the base forecast DataFrame
df = forecast_df.copy()

# Define numerical and categorical features
numerical_features = [
    'forecasted_demand', 'unit_cost', 'revenue', 
    'lead_time_mean', 'lead_time_std_dev', 'cv_demand', 
    'eoq_units', 'safety_stock_units', 'reorder_point_units'  # Added here
]

categorical_features = [
    'demand_pattern', 'ABC_Class', 'XYZ_Class', 'volatility_class'
]

# Define column transformer for preprocessing
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_features),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
    ]
)

# Prepare the feature matrix
X = df[numerical_features + categorical_features]
X_processed = preprocessor.fit_transform(X)


##  Demand Archetype Clustering with MiniBatchKMeans

This script segments 439,000 demand profiles into manageable clusters for downstream analysis or simulation.

###  Objective
Group SKUs into **2000 demand archetypes** using `MiniBatchKMeans` for scalable clustering.

### Key Steps
1. Set `n_clusters = 2000` to define desired segment count
2. Use `MiniBatchKMeans` for memory-efficient clustering
   - `batch_size=2048` for faster convergence
   - `n_init='auto'` to auto-select initialization strategy
3. Assign resulting `cluster_id` to each row in `df`




In [68]:
from sklearn.cluster import MiniBatchKMeans

# Set your target number of clusters
n_clusters = 2000

# Initialize and run the clustering algorithm
kmeans = MiniBatchKMeans(
    n_clusters=n_clusters,
    random_state=42,
    batch_size=2048,  
    n_init='auto'
)

# Assign a cluster label to each of the 439k rows
df['cluster_id'] = kmeans.fit_predict(X_processed)

print(f"Successfully clustered 439k rows into {n_clusters} demand archetypes.")



Successfully clustered 439k rows into 2000 demand archetypes.


## Clustered Demand Profile Aggregation for AnyLogistix

This script generates a **pre-simualtion dataset** by aggregating clustered SKU-level data into archetype-level demand profiles. This will be processed for final anylogistix tables.

### 🔹 Purpose
To convert 439k+ SKU-level rows into **2000 representative demand archetypes**, suitable for scalable supply chain simulation in **AnyLogistix**.

---

### 🔹 Aggregation Logic

- **Categorical Columns**: Mode (`x.mode()[0]`)
- **Key Identifiers**: First or mode
- **Demand, Revenue, Unit Cost**: Summed
- **Other Numerical Fields**: Mean

---

### 🔹 Output: `simulation_demand_df`

| Field            | Description                                |
|------------------|--------------------------------------------|
| `Customer_ID`    | Cluster ID (e.g., `Demand_Archetype_12`)   |
| `Product`        | Dominant part type in cluster              |
| `Location`       | Most frequent location in cluster          |
| `Demand`         | Total forecasted demand for the cluster    |
| `Revenue`        | Total revenue for the cluster              |
| `ReorderPoint`   | Average reorder point for the cluster      |
| ...              | Additional aggregated features             |

---




In [70]:
import pandas as pd

# Mode function for categorical variables
mode_agg = lambda x: x.mode()[0] if not x.mode().empty else 'N/A'

# Define column categories
categorical_cols = [
    'part_type', 'region', 'method_used', 'demand_pattern',
    'replenishment_strategy', 'ABC_Class', 'XYZ_Class',
    'volume_class', 'volatility_class', 'part_prefix'
]
identifier_cols = ['part_id', 'location_id', 'customer_id']
date_cols = ['forecast_date_str']
excluded_cols = categorical_cols + identifier_cols + date_cols + ['cluster_id']

# Detect valid numerical columns automatically
numerical_cols = [
    col for col in df.columns
    if col not in excluded_cols and pd.api.types.is_numeric_dtype(df[col])
]

# Build aggregation dictionary
agg_dict = {
    'cluster_id': 'first',
    'part_id': mode_agg,
    'part_type': mode_agg,
    'location_id': mode_agg,
    'customer_id': mode_agg,
    'forecast_date': mode_agg
}

# Mode aggregation for categorical columns
agg_dict.update({col: mode_agg for col in categorical_cols})

# Sum aggregation for demand, revenue, unit_cost
agg_dict.update({
    'forecasted_demand': 'sum',
    'revenue': 'sum',
    'unit_cost': 'sum'
})

# Mean aggregation for rest of numerical columns
agg_dict.update({
    col: 'mean' for col in numerical_cols if col not in ['forecasted_demand', 'revenue', 'unit_cost']
})

# Final grouped summary
simulation_demand_df = df.groupby('cluster_id').agg(agg_dict).reset_index(drop=True)

# Rename columns for AnyLogistix compatibility
simulation_demand_df.rename(columns={
    'cluster_id': 'Customer_ID',
    'part_type': 'Product',
    'location_id': 'Location',
    'forecasted_demand': 'Demand',
    'revenue': 'Revenue',
    'reorder_point_units': 'ReorderPoint'
}, inplace=True)

# Format Customer_ID as simulation-friendly string
simulation_demand_df['Customer_ID'] = simulation_demand_df['Customer_ID'].apply(
    lambda x: f"Demand_Archetype_{x}"
)

# Reorder key columns
priority_cols = ['Customer_ID', 'Product', 'Location', 'Demand', 'Revenue', 'ReorderPoint']
remaining_cols = [col for col in simulation_demand_df.columns if col not in priority_cols]
simulation_demand_df = simulation_demand_df[priority_cols + remaining_cols]

print("\nFinal simulation demand profile ready for AnyLogistix:")
simulation_demand_df


Final simulation demand profile ready for AnyLogistix:


,Customer_ID,Product,Location,Demand,Revenue,ReorderPoint,part_id,customer_id,forecast_date,region,method_used,demand_pattern,replenishment_strategy,ABC_Class,XYZ_Class,volume_class,volatility_class,part_prefix,unit_cost,forecast_lower_bound,forecast_upper_bound,mae,rmse,mase,rmsse,bias,avg_cost_impact,base_lead_time,adjusted_lead_time,lead_time_mean,lead_time_std_dev,lead_time_min,lead_time_max,lead_time_p90,lead_time_p95,service_level,z_value,ordering_cost_per_order,holding_cost_rate,stockout_cost_per_unit,holding_cost_per_unit,demand_variance_during_lt,safety_stock_units,expected_demand_during_lt,annual_demand_units,cv_demand,intermittent_adjustment,eoq_units,cycle_stock_units,avg_inventory_level_units,abc_xyz_safety_multiplier,safety_stock_units_adjusted,avg_inventory_level_units_adjusted,expected_shortage_per_cycle,fill_rate,days_of_supply,achieved_service_level
0,Demand_Archetype_0,BRAKE_PAD_TYPE_3,TX_3,78.713416,145.89,3.050345,LED_PANEL_2_003,CUST_CACA1BP3_LUM_CZ_20140531_BRA_134859,2013-12-26,TX,TSB,Lumpy,Project-Based,C,Z,Low Volume,Variable,BRA,529.38,0.276187,0.414281,0.409222,0.580010,0.854501,0.666671,0.008886,2.459693,3.724649,4.354430,4.354430,1.088026,2.610570,8.709737,5.661009,6.532105,0.90,1.281552,400.0,0.220000,8.798289,0.510805,1.534246,1.549387,1.500958,126.010512,1.679046,1.839523,821.014935,410.507467,412.056854,1.0,1.549387,412.056854,0.212176,0.858665,1223.984208,0.95
1,Demand_Archetype_1,BRAKE_PAD_TYPE_3,TX_3,163.022223,66.80,2.556248,BRAKE_PAD_3_793,CUST_CACA2BP2_LUM_BZ_20160220_BRA_82083,2014-03-05,TX,TSB,Lumpy,Project-Based,B,Z,Medium Volume,Variable,BRA,1657.97,0.228402,0.342604,0.518082,0.720194,0.818758,0.669996,0.004749,3.912750,2.709387,3.158511,3.158511,0.789825,1.894746,6.317338,4.104553,4.736200,0.95,1.644854,600.0,0.220000,13.023975,0.638798,1.064008,1.655701,0.900547,104.208602,1.987680,1.993840,877.066763,438.533381,440.189083,1.2,1.986841,440.520223,0.103816,0.884579,1588.360476,0.95
2,Demand_Archetype_2,FAN_MOTOR_TYPE_1,CA_2,2976.848144,2715.71,37.485695,FAN_MOTOR_1_339,CUST_CACA2FM1_ERR_AY_20120712_FAN_252945,2013-11-20,CA,SARIMAX,Erratic,Dynamic ROP,A,Y,High Volume,Highly Stable,FAN,440.58,5.199735,7.799602,3.513053,4.963094,0.817706,0.794110,0.112837,8.559956,2.845459,3.057860,3.057860,0.764978,1.834563,6.116332,3.975568,4.587118,0.98,2.053749,800.0,0.158472,3.061004,0.151968,75.271117,17.757732,19.727963,2372.378979,0.777670,1.388835,6921.455029,3460.727514,3478.485247,1.0,17.757732,3478.485247,0.418647,0.978356,542.675435,0.95
3,Demand_Archetype_3,LED_PANEL_TYPE_2,CA_1,78.282961,6.76,1.929434,FAN_MOTOR_2_362,CUST_CACA1BP3_LUM_CZ_20140629_BRA_134888,2014-11-12,CA,TSB,Lumpy,Project-Based,C,Z,Low Volume,Highly Volatile,LED,1509.76,0.147010,0.220515,0.287628,0.474904,0.889438,0.666763,0.008875,2.587676,3.856056,4.892676,4.892676,1.223005,2.936667,9.787066,6.360657,7.339883,0.90,1.281552,400.0,0.250000,11.222582,0.886009,0.691186,1.030685,0.898749,67.073429,1.982922,1.991461,487.965837,243.982918,245.013603,1.0,1.030685,245.013603,0.141144,0.842641,1375.874899,0.95
4,Demand_Archetype_4,FAN_MOTOR_TYPE_1,WI_2,60.096001,0.00,4.813694,FAN_MOTOR_1_255,CUST_CACA1FM2_LUM_AZ_20150101_FAN_325968,2014-12-05,WI,TSB,Lumpy,Project-Based,A,Z,Medium Volume,Variable,FAN,1325.51,0.312187,0.468281,0.590258,0.825905,0.781271,0.663192,0.000264,13.087468,3.646688,4.239091,4.239091,1.060649,2.542468,8.478831,5.511494,6.357338,0.98,2.053749,800.0,0.220000,25.251623,1.893586,2.516772,3.156186,1.657508,142.435326,1.922535,1.961267,675.866991,337.933496,341.089682,1.3,4.103042,342.036538,0.074409,0.954745,908.797543,0.95
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,Demand_Archetype_1995,BRAKE_PAD_TYPE_2,CA_1,523.696929,3796.51,12.136434,BRAKE_PAD_2_183,CUST_CACA1BP2_LUM_AZ_20121007_BRA_56369,2011-02-09,CA,TSB,Lumpy,Proje

### Merge with `simulation_demand_df` on Location

In [73]:
# Merge with simulation_demand_df on Location
simulation_demand_df = simulation_demand_df.merge(location_coordinates, on='Location', how='left')

cols_priority = ['Customer_ID', 'Product', 'Location', 'City', 'Latitude', 'Longitude', 'Demand', 'Revenue']
remaining_cols = [col for col in simulation_demand_df.columns if col not in cols_priority]
simulation_demand_df = simulation_demand_df[cols_priority + remaining_cols]


print("Final simulation demand profile with coordinates mapped:")
simulation_demand_df


Final simulation demand profile with coordinates mapped:


,Customer_ID,Product,Location,City,Latitude,Longitude,Demand,Revenue,ReorderPoint,part_id,customer_id,forecast_date,region,method_used,demand_pattern,replenishment_strategy,ABC_Class,XYZ_Class,volume_class,volatility_class,part_prefix,unit_cost,forecast_lower_bound,forecast_upper_bound,mae,rmse,mase,rmsse,bias,avg_cost_impact,base_lead_time,adjusted_lead_time,lead_time_mean,lead_time_std_dev,lead_time_min,lead_time_max,lead_time_p90,lead_time_p95,service_level,z_value,ordering_cost_per_order,holding_cost_rate,stockout_cost_per_unit,holding_cost_per_unit,demand_variance_during_lt,safety_stock_units,expected_demand_during_lt,annual_demand_units,cv_demand,intermittent_adjustment,eoq_units,cycle_stock_units,avg_inventory_level_units,abc_xyz_safety_multiplier,safety_stock_units_adjusted,avg_inventory_level_units_adjusted,expected_shortage_per_cycle,fill_rate,days_of_supply,achieved_service_level
0,Demand_Archetype_0,BRAKE_PAD_TYPE_3,TX_3,"Austin, TX",30.2711,-97.7437,78.713416,145.89,3.050345,LED_PANEL_2_003,CUST_CACA1BP3_LUM_CZ_20140531_BRA_134859,2013-12-26,TX,TSB,Lumpy,Project-Based,C,Z,Low Volume,Variable,BRA,529.38,0.276187,0.414281,0.409222,0.580010,0.854501,0.666671,0.008886,2.459693,3.724649,4.354430,4.354430,1.088026,2.610570,8.709737,5.661009,6.532105,0.90,1.281552,400.0,0.220000,8.798289,0.510805,1.534246,1.549387,1.500958,126.010512,1.679046,1.839523,821.014935,410.507467,412.056854,1.0,1.549387,412.056854,0.212176,0.858665,1223.984208,0.95
1,Demand_Archetype_1,BRAKE_PAD_TYPE_3,TX_3,"Austin, TX",30.2711,-97.7437,163.022223,66.80,2.556248,BRAKE_PAD_3_793,CUST_CACA2BP2_LUM_BZ_20160220_BRA_82083,2014-03-05,TX,TSB,Lumpy,Project-Based,B,Z,Medium Volume,Variable,BRA,1657.97,0.228402,0.342604,0.518082,0.720194,0.818758,0.669996,0.004749,3.912750,2.709387,3.158511,3.158511,0.789825,1.894746,6.317338,4.104553,4.736200,0.95,1.644854,600.0,0.220000,13.023975,0.638798,1.064008,1.655701,0.900547,104.208602,1.987680,1.993840,877.066763,438.533381,440.189083,1.2,1.986841,440.520223,0.103816,0.884579,1588.360476,0.95
2,Demand_Archetype_2,FAN_MOTOR_TYPE_1,CA_2,"San Diego, CA",32.7174,-117.1628,2976.848144,2715.71,37.485695,FAN_MOTOR_1_339,CUST_CACA2FM1_ERR_AY_20120712_FAN_252945,2013-11-20,CA,SARIMAX,Erratic,Dynamic ROP,A,Y,High Volume,Highly Stable,FAN,440.58,5.199735,7.799602,3.513053,4.963094,0.817706,0.794110,0.112837,8.559956,2.845459,3.057860,3.057860,0.764978,1.834563,6.116332,3.975568,4.587118,0.98,2.053749,800.0,0.158472,3.061004,0.151968,75.271117,17.757732,19.727963,2372.378979,0.777670,1.388835,6921.455029,3460.727514,3478.485247,1.0,17.757732,3478.485247,0.418647,0.978356,542.675435,0.95
3,Demand_Archetype_3,LED_PANEL_TYPE_2,CA_1,"Los Angeles, CA",34.0537,-118.2428,78.282961,6.76,1.929434,FAN_MOTOR_2_362,CUST_CACA1BP3_LUM_CZ_20140629_BRA_134888,2014-11-12,CA,TSB,Lumpy,Project-Based,C,Z,Low Volume,Highly Volatile,LED,1509.76,0.147010,0.220515,0.287628,0.474904,0.889438,0.666763,0.008875,2.587676,3.856056,4.892676,4.892676,1.223005,2.936667,9.787066,6.360657,7.339883,0.90,1.281552,400.0,0.250000,11.222582,0.886009,0.691186,1.030685,0.898749,67.073429,1.982922,1.991461,487.965837,243.982918,245.013603,1.0,1.030685,245.013603,0.141144,0.842641,1375.874899,0.95
4,Demand_Archetype_4,FAN_MOTOR_TYPE_1,WI_2,"Madison, WI",43.0748,-89.3838,60.096001,0.00,4.813694,FAN_MOTOR_1_255,CUST_CACA1FM2_LUM_AZ_20150101_FAN_325968,2014-12-05,WI,TSB,Lumpy,Project-Based,A,Z,Medium Volume,Variable,FAN,1325.51,0.312187,0.468281,0.590258,0.825905,0.781271,0.663192,0.000264,13.087468,3.646688,4.239091,4.239091,1.060649,2.542468,8.478831,5.511494,6.357338,0.98,2.053749,800.0,0.220000,25.251623,1.893586,2.516772,3.156186,1.657508,142.435326,1.922535,1.961267,675.866991,337.933496,341.089682,1.3,4.103042,342.036538,0.074409,0.954745,908.797543,0.95
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,..

##  Super-Customer Grouping for Simulation Scalability

This script clusters demand archetypes into **100 super-customers** for higher-level aggregation, aiding simulation and strategic analysis.

---

### 🔹 Objective
Simplify 2000 demand clusters into 100 super-groups (`SuperCustomer_0` to `SuperCustomer_99`) using **KMeans** based on demand, revenue, and unit cost.

---

### 🔹 Process Overview

1. **Feature Selection**:  
   `['Demand', 'Revenue', 'unit_cost']` → used for clustering.

2. **KMeans Clustering**:  
   Segments demand archetypes into `n_clusters = 100`.

3. **Weighted Cost Calculation**:  
   - `demand_weighted_cost = Demand × unit_cost`

4. **Aggregation Logic**:  
   - `Demand`, `Revenue`: sum  
   - `unit_cost`: both simple mean and weighted average  
   - `region`, `City`, `volatility_class`, `ABC_Class`: mode  
   - `Customer_ID`: count of merged clusters

5. **Final Labeling**:  
   Adds `super_customer_name` for AnyLogistix-friendly identifiers.

---

### 🔹 Output: `df_super_customers`

| Column               | Description                                 |
|----------------------|---------------------------------------------|
| `super_customer_id`  | Numeric ID from KMeans                      |
| `super_customer_name`| String label like `SuperCustomer_12`        |
| `total_demand`       | Total demand from all merged clusters       |
| `Revenue`            | Total revenue                               |
| `weighted_unit_cost` | Demand-weighted average cost                |
| `num_clusters_merged`| Number of demand clusters combined          |
| `region`, `City`     | Dominant regional info                      |
| `volatility_class`   | Cluster volatility mode                     |

---

###  Use Case
Use `df_super_customers` as **aggregate demand nodes** for making final customer table

In [74]:
import pandas as pd
from sklearn.cluster import KMeans

df_clusters = simulation_demand_df.copy()
# Step 1: Select features for grouping into super-customers
features = ['Demand', 'Revenue', 'unit_cost']
X = df_clusters[features]

# Step 2: KMeans clustering to get 100 super-customers
kmeans = KMeans(n_clusters=100, random_state=42)
df_clusters['super_customer_id'] = kmeans.fit_predict(X)

# Step 3: Compute weighted cost for later weighted average
df_clusters['demand_weighted_cost'] = df_clusters['Demand'] * df_clusters['unit_cost']

# Step 4: Aggregation logic to form 100 super-customers
agg_dict = {
    'Demand': 'sum',  # Total demand
    'Revenue': 'sum',
    'demand_weighted_cost': 'sum',
    'unit_cost': 'mean',  # Unweighted average (for comparison)
    'Customer_ID': 'count',
    'region': lambda x: x.mode().iloc[0] if not x.mode().empty else 'Mixed',
    'City': lambda x: x.mode().iloc[0] if not x.mode().empty else 'Mixed',
    'volatility_class': lambda x: x.mode().iloc[0] if not x.mode().empty else 'Mixed',
    'ABC_Class': lambda x: x.mode().iloc[0] if not x.mode().empty else 'Mixed'
}

df_super_customers = df_clusters.groupby('super_customer_id').agg(agg_dict).reset_index()
df_super_customers = df_super_customers.rename(columns={
    'Demand': 'total_demand',
    'Customer_ID': 'num_clusters_merged'
})

# Step 5: Compute weighted unit cost
df_super_customers['weighted_unit_cost'] = (
    df_super_customers['demand_weighted_cost'] / df_super_customers['total_demand']
)

# Step 6: Add final simulation labels
df_super_customers['super_customer_name'] = 'SuperCustomer_' + df_super_customers['super_customer_id'].astype(str)

# Step 7: Final cleanup (drop demand_weighted_cost if not needed)
df_super_customers.drop(columns=['demand_weighted_cost'], inplace=True)


df_super_customers


,super_customer_id,total_demand,Revenue,unit_cost,num_clusters_merged,region,City,volatility_class,ABC_Class,weighted_unit_cost,super_customer_name
0,0,6669.307176,34542.75,663.737619,42,CA,"Dallas, TX",Stable,B,652.412848,SuperCustomer_0
1,1,1472.622744,39520.31,2328.722500,4,CA,"Los Angeles, CA",Stable,A,2285.712257,SuperCustomer_1
2,2,3293.613766,703.55,2013.354167,48,TX,"Dallas, TX",Highly Volatile,A,2020.254316,SuperCustomer_2
3,3,7214.963411,28756.84,736.377143,7,WI,"Austin, TX",Highly Stable,A,703.937108,SuperCustomer_3
4,4,379.900102,0.00,4473.854444,9,TX,"Dallas, TX",Highly Volatile,A,4451.686437,SuperCustomer_4
...,...,...,...,...,...,...,...,...,...,...,...
95,95,3393.281778,10168.30,1246.590000,2,CA,"Austin, TX",Highly Stable,A,1272.349124,SuperCustomer_95
96,96,2346.007666,643.96,2610.169688,32,CA,"Dallas, TX",Highly Volatile,A,2602.379710,SuperCustomer_96
97,97,2722.465865,28377.05,1525.889412,17,CA,"Dallas, TX",Stable,A,1543.531856,SuperCustomer_97
98,98,8575.164499,6376.28,194.375000,2,CA,"San Jose, CA",Highly Stable,A,195.285278,SuperCustomer_98


### Super customer ID creation

In [75]:
simulation_demand_df['super_customer_id'] = 'SuperCustomer_' + df_clusters['super_customer_id'].astype(str)

In [76]:
simulation_demand_df

,Customer_ID,Product,Location,City,Latitude,Longitude,Demand,Revenue,ReorderPoint,part_id,customer_id,forecast_date,region,method_used,demand_pattern,replenishment_strategy,ABC_Class,XYZ_Class,volume_class,volatility_class,part_prefix,unit_cost,forecast_lower_bound,forecast_upper_bound,mae,rmse,mase,rmsse,bias,avg_cost_impact,base_lead_time,adjusted_lead_time,lead_time_mean,lead_time_std_dev,lead_time_min,lead_time_max,lead_time_p90,lead_time_p95,service_level,z_value,ordering_cost_per_order,holding_cost_rate,stockout_cost_per_unit,holding_cost_per_unit,demand_variance_during_lt,safety_stock_units,expected_demand_during_lt,annual_demand_units,cv_demand,intermittent_adjustment,eoq_units,cycle_stock_units,avg_inventory_level_units,abc_xyz_safety_multiplier,safety_stock_units_adjusted,avg_inventory_level_units_adjusted,expected_shortage_per_cycle,fill_rate,days_of_supply,achieved_service_level,super_customer_id
0,Demand_Archetype_0,BRAKE_PAD_TYPE_3,TX_3,"Austin, TX",30.2711,-97.7437,78.713416,145.89,3.050345,LED_PANEL_2_003,CUST_CACA1BP3_LUM_CZ_20140531_BRA_134859,2013-12-26,TX,TSB,Lumpy,Project-Based,C,Z,Low Volume,Variable,BRA,529.38,0.276187,0.414281,0.409222,0.580010,0.854501,0.666671,0.008886,2.459693,3.724649,4.354430,4.354430,1.088026,2.610570,8.709737,5.661009,6.532105,0.90,1.281552,400.0,0.220000,8.798289,0.510805,1.534246,1.549387,1.500958,126.010512,1.679046,1.839523,821.014935,410.507467,412.056854,1.0,1.549387,412.056854,0.212176,0.858665,1223.984208,0.95,SuperCustomer_41
1,Demand_Archetype_1,BRAKE_PAD_TYPE_3,TX_3,"Austin, TX",30.2711,-97.7437,163.022223,66.80,2.556248,BRAKE_PAD_3_793,CUST_CACA2BP2_LUM_BZ_20160220_BRA_82083,2014-03-05,TX,TSB,Lumpy,Project-Based,B,Z,Medium Volume,Variable,BRA,1657.97,0.228402,0.342604,0.518082,0.720194,0.818758,0.669996,0.004749,3.912750,2.709387,3.158511,3.158511,0.789825,1.894746,6.317338,4.104553,4.736200,0.95,1.644854,600.0,0.220000,13.023975,0.638798,1.064008,1.655701,0.900547,104.208602,1.987680,1.993840,877.066763,438.533381,440.189083,1.2,1.986841,440.520223,0.103816,0.884579,1588.360476,0.95,SuperCustomer_92
2,Demand_Archetype_2,FAN_MOTOR_TYPE_1,CA_2,"San Diego, CA",32.7174,-117.1628,2976.848144,2715.71,37.485695,FAN_MOTOR_1_339,CUST_CACA2FM1_ERR_AY_20120712_FAN_252945,2013-11-20,CA,SARIMAX,Erratic,Dynamic ROP,A,Y,High Volume,Highly Stable,FAN,440.58,5.199735,7.799602,3.513053,4.963094,0.817706,0.794110,0.112837,8.559956,2.845459,3.057860,3.057860,0.764978,1.834563,6.116332,3.975568,4.587118,0.98,2.053749,800.0,0.158472,3.061004,0.151968,75.271117,17.757732,19.727963,2372.378979,0.777670,1.388835,6921.455029,3460.727514,3478.485247,1.0,17.757732,3478.485247,0.418647,0.978356,542.675435,0.95,SuperCustomer_84
3,Demand_Archetype_3,LED_PANEL_TYPE_2,CA_1,"Los Angeles, CA",34.0537,-118.2428,78.282961,6.76,1.929434,FAN_MOTOR_2_362,CUST_CACA1BP3_LUM_CZ_20140629_BRA_134888,2014-11-12,CA,TSB,Lumpy,Project-Based,C,Z,Low Volume,Highly Volatile,LED,1509.76,0.147010,0.220515,0.287628,0.474904,0.889438,0.666763,0.008875,2.587676,3.856056,4.892676,4.892676,1.223005,2.936667,9.787066,6.360657,7.339883,0.90,1.281552,400.0,0.250000,11.222582,0.886009,0.691186,1.030685,0.898749,67.073429,1.982922,1.991461,487.965837,243.982918,245.013603,1.0,1.030685,245.013603,0.141144,0.842641,1375.874899,0.95,SuperCustomer_32
4,Demand_Archetype_4,FAN_MOTOR_TYPE_1,WI_2,"Madison, WI",43.0748,-89.3838,60.096001,0.00,4.813694,FAN_MOTOR_1_255,CUST_CACA1FM2_LUM_AZ_20150101_FAN_325968,2014-12-05,WI,TSB,Lumpy,Project-Based,A,Z,Medium Volume,Variable,FAN,1325.51,0.312187,0.468281,0.590258,0.825905,0.781271,0.663192,0.000264,13.087468,3.646688,4.239091,4.239091,1.060649,2.542468,8.478831,5.511494,6.357338,0.98,2.053749,800.0,0.220000,25.251623,1.893586,2.516772,3.156186,1.657508,142.435326,1.922535,1.961267,675.866991,337.933496,341.089682,1.3,4.103042,342.036538,0.074409,0.954745,908.797543,0.95,SuperCustomer_74
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...

### part type classification logic

In [77]:
import pandas as pd


def map_part_type(code):
    if "BRAKE_PAD" in code:
        return "BRAKE PAD"
    elif "LED_PANEL" in code:
        return "LED PANEL"
    elif "FAN_MOTOR" in code:
        return "FAN MOTOR"
    else:
        return code 
    
simulation_demand_df['part_type'] = simulation_demand_df['Product'].apply(map_part_type)




##  AnyLogistix Customer Table Generator

This function creates a simulation-ready **Customers table** for AnyLogistix based on super-customer groupings.

---

### 🔹 Purpose
To generate a formatted customer entity file containing:
- Unique customer IDs
- Descriptive names
- City-based locations (mode)
- Type, inclusion flag, and icon

---

### 🔹 Key Steps

1. **City Aggregation**:  
   Get the most frequent (`mode`) `City` per `super_customer_id`.

2. **Customer Entity Info**:  
   - `ID`: Named as `"Customer 1"` to `"Customer 100"`  
   - `Name`: Super customer ID  
   - `Type`: Set to `'Customer'`  
   - `Location`: Dominant city name  
   - `Inclusion Type`: `'Include'`  
   - `Icon`: Set to `116` (AnyLogistix visual)

3. **Final Output Columns**:  
   `['ID', 'Name', 'Type', 'Location', 'Inclusion Type', 'Icon']`

---

### 🔹 Output: `anylogix_customers_table.csv`

A CSV file ready for import into **AnyLogistix → Customers Table**, enabling:
- Scenario visualization by city
- Demand mapping by super-customer nodes


In [78]:
import pandas as pd

def generate_anylogistix_customers_table(simulation_demand_df):
    # Step 1: Compute mode city for each super customer
    city_mode_df = (
        simulation_demand_df.groupby('super_customer_id')['City']
        .agg(lambda x: x.mode().iloc[0] if not x.mode().empty else 'Unknown')
        .reset_index()
        .rename(columns={'City': 'City_Mode'})
    )

    # Step 2: Get unique super_customer_ids
    super_customers = simulation_demand_df[['super_customer_id']].drop_duplicates().reset_index(drop=True)
    super_customers['ID'] = [f'Customer {i+1}' for i in range(len(super_customers))]
    super_customers['Name'] = super_customers['super_customer_id']
    super_customers['Type'] = 'Customer'
    super_customers['Type'] = 'Customer'

    # Step 3: Merge with mode city info
    super_customers = super_customers.merge(city_mode_df, on='super_customer_id', how='left')
    super_customers = super_customers.rename(columns={'City_Mode': 'Location'})

    # Step 4: Add required columns
    super_customers['Inclusion Type'] = 'Include'
    super_customers['Icon'] = 116

    # Reorder columns
    final_columns = ['ID', 'Name', 'Type', 'Location', 'Inclusion Type', 'Icon']
    customer_df = super_customers[final_columns]

    return customer_df

# Generate table
anylogix_customers_table = generate_anylogistix_customers_table(simulation_demand_df)

# Preview
print(anylogix_customers_table.head())

# Optional: Save
anylogix_customers_table.to_csv("anylogistix_customers_table.csv", index=False)


           ID              Name      Type       Location Inclusion Type  Icon
0  Customer 1  SuperCustomer_41  Customer  Green Bay, WI        Include   116
1  Customer 2  SuperCustomer_92  Customer     Dallas, TX        Include   116
2  Customer 3  SuperCustomer_84  Customer     Austin, TX        Include   116
3  Customer 4  SuperCustomer_32  Customer     Dallas, TX        Include   116
4  Customer 5  SuperCustomer_74  Customer     Dallas, TX        Include   116


In [79]:
pd.set_option('display.max_columns',None)

##  AnyLogistix pre-Demand Table Generator

This function generates a **demand input table** for AnyLogistix using previously clustered and aggregated demand data.This will be used for final demand table creation.

---

### 🔹 Objective
Transform `simulation_demand_df` into the **AnyLogistix-compliant demand format**, including:
- Structured demand ID
- Reorder points
- Periodic demand flags
- Revenue and lead time

---

### 🔹 Key Columns

| Column              | Description                                      |
|---------------------|--------------------------------------------------|
| `ID`                | Unique demand row (e.g., `Demand 1`, `Demand 2`) |
| `Customer`          | Demand archetype ID                              |
| `Super Customer`    | Grouped cluster (regional demand node)           |
| `Product`           | Part/Item name                                   |
| `Demand Type`       | Set as `'PeriodicDemand'`                        |
| `Col 6`             | Reorder point units (rounded integer)            |
| `Col 10`            | Demand quantity                                  |
| `Revenue`           | Total revenue from cluster                       |
| `Expected Lead Time`| Average lead time (float, in days)               |
| `Backorder Policy`  | `'AllowedTotal'` to enable backlog               |
| `Inclusion Type`    | `'Include'` to activate in simulation            |

---

### 🔹 Usage
Import `anylogistix_simulation_demand_table.csv` into **AnyLogistix → Demand Table** to simulate demand behavior at a super-customer level with:
- Periodic replenishment
- Clustered product demand
- Strategic backordering



In [80]:

# Pre Transformation for Demand Table 
def generate_anylogistix_demand_table(simulation_demand_df):
    anylogix_df = pd.DataFrame()

    # ID column: Demand 1, Demand 2, ...
    anylogix_df['ID'] = [f'Demand {i+1}' for i in range(len(simulation_demand_df))]

    # Required columns
    anylogix_df['Customer'] = simulation_demand_df['Customer_ID']
    anylogix_df['Super Customer'] = simulation_demand_df['super_customer_id']
    anylogix_df['Product'] = simulation_demand_df['Product']
    anylogix_df['Demand Type'] = 'PeriodicDemand'

    # Demand descriptors
    anylogix_df['Col 1'] = 'First occurrence'
    anylogix_df['Col 2'] = 'Next day after interval'

    # Col 3 = Reorder point units rounded to int
    anylogix_df['Col 3'] = 'Order interval, days'

    anylogix_df['Col 4'] = 'Value'
    anylogix_df['Col 5'] = 'Value'
    anylogix_df['Col 6'] = simulation_demand_df['ReorderPoint'].round().astype(int)
    anylogix_df['Col 7'] = 'Quantity'
    anylogix_df['Col 8'] = 'Value'
    anylogix_df['Col 9'] = 'Value'
    anylogix_df['Col 10'] = simulation_demand_df['Demand']

    # Time Period
    anylogix_df['Time Period'] = '(All periods)'

    # Revenue
    anylogix_df['Revenue'] = simulation_demand_df['Revenue'].round(6)

    # Currency and Lead Time Info
    anylogix_df['Currency'] = 'USD'
    anylogix_df['Expected Lead Time'] = simulation_demand_df['adjusted_lead_time'].round(6)
    anylogix_df['Time Unit'] = 'day'

    # Simulation-specific flags
    anylogix_df['Minimum Split Ratio'] = 1
    anylogix_df['Backorder Policy'] = 'AllowedTotal'
    anylogix_df['Inclusion Type'] = 'Include'

    return anylogix_df

# Generate the formatted demand table
anylogix_demand_table = generate_anylogistix_demand_table(simulation_demand_df)

anylogix_demand_table.to_csv("anylogistix_simulation_demand_table.csv", index=False)



In [81]:
anylogix_demand_table

,ID,Customer,Super Customer,Product,Demand Type,Col 1,Col 2,Col 3,Col 4,Col 5,Col 6,Col 7,Col 8,Col 9,Col 10,Time Period,Revenue,Currency,Expected Lead Time,Time Unit,Minimum Split Ratio,Backorder Policy,Inclusion Type
0,Demand 1,Demand_Archetype_0,SuperCustomer_41,BRAKE_PAD_TYPE_3,PeriodicDemand,First occurrence,Next day after interval,"Order interval, days",Value,Value,3,Quantity,Value,Value,78.713416,(All periods),145.89,USD,4.354430,day,1,AllowedTotal,Include
1,Demand 2,Demand_Archetype_1,SuperCustomer_92,BRAKE_PAD_TYPE_3,PeriodicDemand,First occurrence,Next day after interval,"Order interval, days",Value,Value,3,Quantity,Value,Value,163.022223,(All periods),66.80,USD,3.158511,day,1,AllowedTotal,Include
2,Demand 3,Demand_Archetype_2,SuperCustomer_84,FAN_MOTOR_TYPE_1,PeriodicDemand,First occurrence,Next day after interval,"Order interval, days",Value,Value,37,Quantity,Value,Value,2976.848144,(All periods),2715.71,USD,3.057860,day,1,AllowedTotal,Include
3,Demand 4,Demand_Archetype_3,SuperCustomer_32,LED_PANEL_TYPE_2,PeriodicDemand,First occurrence,Next day after interval,"Order interval, days",Value,Value,2,Quantity,Value,Value,78.282961,(All periods),6.76,USD,4.892676,day,1,AllowedTotal,Include
4,Demand 5,Demand_Archetype_4,SuperCustomer_74,FAN_MOTOR_TYPE_1,PeriodicDemand,First occurrence,Next day after interval,"Order interval, days",Value,Value,5,Quantity,Value,Value,60.096001,(All periods),0.00,USD,4.239091,day,1,AllowedTotal,Include
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,Demand 1996,Demand_Archetype_1995,SuperCustomer_24,BRAKE_PAD_TYPE_2,PeriodicDemand,First occurrence,Next day after interval,"Order interval, days",Value,Value,12,Quantity,Value,Value,523.696929,(All periods),3796.51,USD,2.384615,day,1,AllowedTotal,Include
1996,Demand 1997,Demand_Archetype_1996,SuperCustomer_41,LED_PANEL_TYPE_1,PeriodicDemand,First occurrence,Next day after interval,"Order interval, days",Value,Value,7,Quantity,Value,Value,33.092190,(All periods),203.93,USD,11.921584,day,1,AllowedTotal,Include
1997,Demand 1998,Demand_Archetype_1997,SuperCustomer_90,LED_PANEL_TYPE_2,PeriodicDemand,First occurrence,Next day after interval,"Order interval, days",Value,Value,3,Quantity,Value,Value,53.260806,(All periods),63.72,USD,6.288571,day,1,AllowedTotal,Include
1998,Demand 1999,Demand_Archetype_1998,SuperCustomer_86,BRAKE_PAD_TYPE_2,PeriodicDemand,First occurrence,Next day after interval,"Order interval, days",Value,Value,7,Quantity,Value,Value,62.611313,(All periods),768.21,USD,4.202317,day,1,AllowedTotal,Include


## Simulated Bill of Materials (BOM) Table Generator

This script creates a simplified **BOM (Bill of Materials)** table for use in **AnyLogistix**, based on forecasted product occurrences.

---

### 🔹 Objective
Estimate the number of finished products (as BOMs) by **counting occurrences** in the forecast data.

---

### 🔹 Output Columns

| Column        | Description                                |
|---------------|--------------------------------------------|
| `ID`          | Unique BOM ID (`<Product> BOM`)            |
| `Name`        | BOM Name (`<Product> BOM`)                 |
| `End Product` | Final product/part type                    |
| `Quantity`    | Occurrence count from `forecast_df`        |

---

### 🔹 Use Case
Import the generated `BOM.csv` into **AnyLogistix → Bill of Materials Table** to:
- Simulate simplified product structure
- Link demand to corresponding end items


In [84]:
# Count occurrences of each Product
product_counts = forecast_df['part_type'].value_counts().reset_index()
product_counts.columns = ['End Product', 'Quantity']

# Add BOM ID and Name
product_counts['ID'] = product_counts['End Product'] + ' BOM'
product_counts['Name'] = product_counts['End Product'] + ' BOM'

# Reorder columns
simulated_bom = product_counts[['ID', 'Name', 'End Product', 'Quantity']]
simulated_bom.to_csv("BOM.csv",index=False)
print(simulated_bom.head(5).to_markdown(index=False))

| ID                   | Name                 | End Product      |   Quantity |
|:---------------------|:---------------------|:-----------------|-----------:|
| BRAKE_PAD_TYPE_3 BOM | BRAKE_PAD_TYPE_3 BOM | BRAKE_PAD_TYPE_3 |     122557 |
| BRAKE_PAD_TYPE_2 BOM | BRAKE_PAD_TYPE_2 BOM | BRAKE_PAD_TYPE_2 |      68290 |
| LED_PANEL_TYPE_1 BOM | LED_PANEL_TYPE_1 BOM | LED_PANEL_TYPE_1 |      63156 |
| FAN_MOTOR_TYPE_2 BOM | FAN_MOTOR_TYPE_2 BOM | FAN_MOTOR_TYPE_2 |      61154 |
| FAN_MOTOR_TYPE_1 BOM | FAN_MOTOR_TYPE_1 BOM | FAN_MOTOR_TYPE_1 |      56729 |


##  Final AnyLogistix Demand Table (Grouped by Super Customer & Product)

This script generates the **final aggregated demand table** (`DEM1.csv`) for simulation in AnyLogistix. It groups demand data by **Super Customer** and **Product**, consolidating multiple entries into a clean, simulation-optimized format.

---

### Aggregation Logic

| Column Type      | Aggregation Method |
|------------------|--------------------|
| `Col 6` (ROP)    | Mean → Rounded int |
| `Col 10` (Demand)| Sum                |
| `Revenue`        | Sum                |
| `Expected Lead Time` | Mean          |
| Categorical cols | Mode               |

---

###  Final Output Columns

| Column                | Description                                |
|------------------------|--------------------------------------------|
| `ID`                   | Aggregated demand ID                       |
| `Super Customer`       | Grouped demand cluster                     |
| `Product`              | End item                                   |
| `Col 6`                | Rounded reorder point                      |
| `Col 10`               | Total demand quantity                      |
| `Revenue`              | Aggregated revenue                         |
| `Expected Lead Time`   | Avg. lead time (days)                      |
| `Demand Type`, `Col 1`–`Col 5`, etc. | Preserved from base table |

---

### Usage
Import `DEM1.csv` into **AnyLogistix → Demand Table** to simulate periodic demand by **super customers** with consolidated demand quantities and financials.



In [ ]:
df3 = anylogix_demand_table.copy()

# Custom mode function
def mode_agg(series):
    modes = series.mode()
    return modes.iloc[0] if not modes.empty else None

# Separate columns
numeric_cols = df3.select_dtypes(include='number').columns.tolist()
non_numeric_cols = [col for col in df3.columns if col not in numeric_cols and col != 'Customer']

# Aggregation: sum all numeric except Col 6 (reorder point) which should be mean
agg_dict1 = {
    col: 'mean' if col in ['Col 6', 'Expected Lead Time'] else 'sum'
    for col in numeric_cols
}
agg_dict1.update({col: mode_agg for col in non_numeric_cols})

# Group and aggregate
dem_11 = df3.groupby(['Super Customer', 'Product'], as_index=False).agg(agg_dict1)
dem_11['Col 6'] = dem_11['Col 6'].round().astype(int)
# Final column order as specified
final_column_order = [
    'ID', 'Super Customer', 'Product', 'Demand Type',
    'Col 1', 'Col 2', 'Col 3', 'Col 4', 'Col 5', 'Col 6',
    'Col 7', 'Col 8', 'Col 9', 'Col 10', 'Time Period',
    'Revenue', 'Currency', 'Expected Lead Time', 'Time Unit',
    'Minimum Split Ratio', 'Backorder Policy', 'Inclusion Type'
]

# Reorder columns
dem_11 = dem_11[final_column_order]

# Save output
dem_11.to_csv("DEM1.csv", index=False)


##  AnyLogistix Distributor Table

This table defines **Distribution Centers (DCs)** for use in AnyLogistix based on the total simulated demand per city. Each DC is automatically assigned:

- A **20% buffer** on demand to determine its `Capacity`
- A unique **ID**, **Name**, and fixed **Icon (114)**
- Standardized fields for AnyLogistix simulation

---

###  Key Columns

| Column Name                  | Description                                |
|------------------------------|--------------------------------------------|
| `ID`                         | Unique identifier for each DC              |
| `Name`                       | Readable name (e.g., DC_Dallas)            |
| `Type`                       | Always `DC`                                |
| `Location`                   | City (based on simulation demand)          |
| `Capacity`                   | Total demand × 1.2 (rounded)               |
| `Initially Open`             | All set to TRUE                            |
| `Inclusion Type`             | All set to Consider                        |
| `Aggregate Orders by Location` | FALSE for independent routing           |
| `Priority`                   | Set to EQUAL                               |

---

###  Output

Saved to: `anylogistix_distributor_table_formatted.csv`  
Format is ready to import into **AnyLogistix → Distributor Table**.


In [ ]:
import pandas as pd

# Step 1: Aggregate demand per location
location_capacity = simulation_demand_df.groupby('City')['Demand'].sum().reset_index()
location_capacity.columns = ['Location', 'Total_Demand']

# Step 2: Apply 20% buffer and round capacity
location_capacity['Capacity'] = (location_capacity['Total_Demand'] * 1.2).round(2)

# Step 3: Build final distributor table
distributor_table = pd.DataFrame({
    'ID': [f'Distribution Center {i+1}' for i in range(len(location_capacity))],
    'Name': [f'DC_{loc}' for loc in location_capacity['Location']],
    'Type': ['DC'] * len(location_capacity),
    'Location': location_capacity['Location'],
    'Initially Open': ['TRUE'] * len(location_capacity),
    'Inclusion Type': ['Consider'] * len(location_capacity),
    'Capacity': location_capacity['Capacity'],
    'Capacity Unit': ['m³'] * len(location_capacity),
    'Priority': ['EQUAL'] * len(location_capacity),
    'Aggregate Orders by Location': ['FALSE'] * len(location_capacity),
    'Icon': [114] * len(location_capacity)
})


print(distributor_table.head().to_markdown(index=False))


distributor_table.to_csv("anylogistix_distributor_table_formatted.csv", index=False)


| ID                    | Name               | Type   | Location        | Initially Open   | Inclusion Type   |   Capacity | Capacity Unit   | Priority   | Aggregate Orders by Location   |   Icon |
|:----------------------|:-------------------|:-------|:----------------|:-----------------|:-----------------|-----------:|:----------------|:-----------|:-------------------------------|-------:|
| Distribution Center 1 | DC_Austin, TX      | DC     | Austin, TX      | TRUE             | Consider         |    88520.6 | m³              | EQUAL      | FALSE                          |    114 |
| Distribution Center 2 | DC_Dallas, TX      | DC     | Dallas, TX      | TRUE             | Consider         |    78317.5 | m³              | EQUAL      | FALSE                          |    114 |
| Distribution Center 3 | DC_Green Bay, WI   | DC     | Green Bay, WI   | TRUE             | Consider         |    39763.4 | m³              | EQUAL      | FALSE                          |    114 |
| Distribu

##  Inventory Stock Policy Table (Product Level)

This table presents calculated inventory control parameters for each unique `Product` across super customers, based on historical demand variability and lead time uncertainty.

The policy follows a **continuous review (Q,R) model** with a service level of **95% (Z = 1.645)**.

---

###  Methodology Summary

- **Daily Demand** = Total Demand / Time Period
- **Safety Stock** = `Z × σ(LT Demand)`  
- **σ(LT Demand)** = √(μ<sub>LT</sub> × σ²<sub>D</sub> + μ²<sub>D</sub> × σ²<sub>LT</sub>)
- **Reorder Point (ROP)** = Min Stock + Safety Stock
- **Max Stock** = ROP

---

###  Output Columns

| Column Name                 | Description                                                   |
|----------------------------|---------------------------------------------------------------|
| `Product`                  | Part type or SKU                                              |
| `Super_Customer`           | Super customer grouping (from clustering)                     |
| `total_demand`             | Total aggregated demand over all periods                      |
| `daily_demand`             | Aggregate daily demand across time windows                    |
| `mean_daily_demand`        | Mean of daily demand per product                              |
| `std_daily_demand`         | Standard deviation of daily demand                            |
| `cv_demand`                | Coefficient of variation for demand                           |
| `avg_lead_time`            | Mean lead time in days                                        |
| `std_lead_time`            | Standard deviation of lead time                               |
| `cv_lead_time`             | Coefficient of variation for lead time                        |
| `Safety Stock`             | Buffer stock to meet service level                            |
| `Min Stock`                | Expected stock during lead time                               |
| `Reorder Point`            | Trigger level for replenishment                               |
| `Max Stock`                | Upper limit for inventory holding                             |

---




In [ ]:
import pandas as pd
import numpy as np

def calculate_inventory_stocks_by_product(df, service_level_z=1.645):
    """
    Calculates inventory stock levels (Min, Max, Safety, Reorder Point) for each Product 
    based on grouped demand and lead time variability.

    Args:
        df (pd.DataFrame): Raw transactional or historical demand data.
        service_level_z (float): Z-score corresponding to desired service level (default=1.645 for 95%).

    Returns:
        pd.DataFrame: Inventory policy table at Product level.
    """

    # --- Ensure numeric conversion ---
    df['demand'] = pd.to_numeric(df['Col 10'], errors='coerce')   # Demand
    df['lead_time_days'] = pd.to_numeric(df['Expected Lead Time'], errors='coerce')  # Lead time

 
    df['time_period_days'] =df['Col 6']
        # --- Filter out invalid time periods (0 or NaN) ---
    df = df[df['time_period_days'] > 0].copy()
    df['daily_demand']=df['demand']/df['time_period_days']
  
    # --- Group by Product ---
    grouped = df.groupby('Product').agg(
        total_demand=('demand', 'sum'),
        daily_demand=('daily_demand', 'sum'),
        avg_time_period=('time_period_days',"mean"),
        mean_daily_demand=('daily_demand', 'mean'),
        std_daily_demand=('daily_demand', 'std'),
        avg_lead_time=('lead_time_days', 'mean'),
        std_lead_time=('lead_time_days', 'std'),
        Super_Customer=('Super Customer', 'first'),  # keep any one customer value for reporting
        ID=('ID', 'first')  # same for ID
    ).reset_index()

    # Calculate Coefficients of Variation
    grouped['cv_demand'] = grouped['std_daily_demand'] / grouped['mean_daily_demand']
    grouped['cv_lead_time'] = grouped['std_lead_time'] / grouped['avg_lead_time']

    # --- Safety Stock Calculation ---
    variance_demand_during_lt = (grouped['avg_lead_time'] * grouped['std_daily_demand']**2) + \
                                 (grouped['mean_daily_demand']**2 * grouped['std_lead_time']**2)

    grouped['std_dev_demand_during_lt'] = np.sqrt(variance_demand_during_lt)
    grouped['Safety Stock'] = service_level_z * grouped['std_dev_demand_during_lt']

    # --- Final Inventory Levels ---
    grouped['Min Stock'] = grouped['daily_demand'] * grouped['avg_lead_time']
    grouped['Max Stock'] = grouped['Min Stock'] + grouped['Safety Stock']
    grouped['Reorder Point'] = grouped['Min Stock'] + grouped['Safety Stock']

    output_cols = [
        'ID', 'Super_Customer', 'Product','avg_time_period',
        'total_demand','daily_demand',
        'mean_daily_demand', 'std_daily_demand', 'cv_demand',
        'avg_lead_time', 'std_lead_time', 'cv_lead_time',
        'std_dev_demand_during_lt',
        'Min Stock', 'Safety Stock', 'Max Stock', 'Reorder Point'
    ]

    result_df = grouped[output_cols].round(2)
    return result_df

    


inventory_policy = calculate_inventory_stocks_by_product(dem_11)
inventory_policy


,ID,Super_Customer,Product,avg_time_period,total_demand,daily_demand,mean_daily_demand,std_daily_demand,cv_demand,avg_lead_time,std_lead_time,cv_lead_time,std_dev_demand_during_lt,Min Stock,Safety Stock,Max Stock,Reorder Point
0,Demand 1256,SuperCustomer_13,BRAKE_PAD_TYPE_1,9.61,21084.97,2189.04,70.61,51.95,0.74,3.15,0.53,0.17,99.45,6905.46,163.59,7069.05,7069.05
1,Demand 1331,SuperCustomer_0,BRAKE_PAD_TYPE_2,14.81,81824.32,7347.18,106.48,86.22,0.81,2.92,0.54,0.19,158.20,21420.80,260.24,21681.04,21681.04
2,Demand 1109,SuperCustomer_0,BRAKE_PAD_TYPE_3,20.60,224356.25,21088.69,324.44,320.75,0.99,2.94,0.35,0.12,561.54,62035.60,923.73,62959.33,62959.33
3,Demand 1474,SuperCustomer_0,FAN_MOTOR_TYPE_1,17.52,81697.21,6019.52,89.84,71.25,0.79,3.64,0.82,0.23,154.67,21928.05,254.43,22182.48,22182.48
4,Demand 1853,SuperCustomer_0,FAN_MOTOR_TYPE_2,4.78,24124.14,6587.35,146.39,131.75,0.90,4.24,0.89,0.21,301.04,27909.38,495.20,28404.58,28404.58
5,Demand 1210,SuperCustomer_0,LED_PANEL_TYPE_1,9.02,67451.00,6925.80,125.92,113.12,0.90,5.49,1.15,0.21,302.30,38053.72,497.28,38551.00,38551.00
6,Demand 1050,SuperCustomer_0,LED_PANEL_TYPE_2,3.61,11450.78,3554.97,154.56,154.19,1.00,6.52,1.03,0.16,424.65,23188.66,698.55,23887.20,23887.20


##  Final Inventory Policy Table (AnyLogistix Format)

This table converts SKU-level inventory planning parameters into the format expected by **AnyLogistix** for simulation and execution of inventory policies. It uses the **Min-Max-Safety Stock** logic with periodic review.

---

###  Key Settings:

| Field                     | Value/Logic                                                  |
|--------------------------|--------------------------------------------------------------|
| `Policy Type`            | `InventoryPolicyMinMaxSafetyStock`                          |
| `Initial Stock, units`   | `Min Stock + Safety Stock` (rounded to 2 decimals)           |
| `First Periodic Check`   | Starts from `2-June-2011`, one day increment per product     |
| `Facility`               | `(All sites)` (useful when abstracted facility data)         |
| `Periodic Check`         | Enabled (`TRUE`)                                             |
| `Time Unit`              | Day                                                          |

---

###  Columns Description:

| Column Name              | Description                                                      |
|--------------------------|------------------------------------------------------------------|
| `ID`                     | Unique identifier (Inventory 1, Inventory 2, …)                 |
| `Facility`               | Where policy applies (default: All sites)                       |
| `Product`                | SKU / Part Number                                                |
| `Policy Type`            | Strategy used (Min-Max-SafetyStock based)                       |
| `Col 1` to `Col 6`       | Structured values (Max, Min, Safety Stock and their values)     |
| `Initial Stock, units`   | Starting stock for simulation                                    |
| `Periodic Check`         | Whether inventory is reviewed periodically                       |
| `Period`                 | Review frequency (set to 1 day)                                 |
| `First Periodic Check`   | When review begins                                               |
| `Policy Basis`           | Quantity-based policy                                            |
| `Stock Calculation Window`| Set to 0 (non-rolling window)                                  |
| `Time Period`            | All periods considered                                           |
| `Inclusion Type`         | Include in simulation                                            |

---




In [ ]:
from datetime import datetime

def format_inventory_policy_table(inventory_df):
    """
    Converts inventory policy DataFrame to formatted policy table as per spec.
    """

    formatted_rows = []

    for i, row in inventory_df.iterrows():
        formatted_rows.append({
            'ID': f'Inventory {i + 1}',
            'Facility': '(All sites)',
            'Product': row['Product'],
            'Policy Type': 'InventoryPolicyMinMaxSafetyStock',
            'Col 1': 'Max',
            'Col 2': row['Max Stock'],
            'Col 3': 'Min',
            'Col 4': row['Min Stock'],
            'Col 5': 'Safety stock',
            'Col 6': row['Safety Stock'],
            'Initial Stock, units': round((row['Min Stock'] + row['Safety Stock']), 2),
            'Periodic Check': 'TRUE',
            'Period': 1,
            'First Periodic Check': datetime(2011, 6, 2 + i).strftime('%#d-%#m-%y 00:00:00') if hasattr(datetime(2011, 6, 2 + i), 'strftime') else f"{2+i}-6-11 00:00:00",
            'Policy Basis': 'Quantity',
            'Stock Calculation Window': 0,
            'Time Unit': 'day',
            'Minimum Split Ratio': 1,
            'Time Period': '(All periods)',
            'Inclusion Type': 'Include'
        })

    return pd.DataFrame(formatted_rows)

policy_table = format_inventory_policy_table(inventory_policy)





policy_table.to_excel("Final_Inventory_Policy_Table.xlsx", index=False)
policy_table

,ID,Facility,Product,Policy Type,Col 1,Col 2,Col 3,Col 4,Col 5,Col 6,"Initial Stock, units",Periodic Check,Period,First Periodic Check,Policy Basis,Stock Calculation Window,Time Unit,Minimum Split Ratio,Time Period,Inclusion Type
0,Inventory 1,(All sites),BRAKE_PAD_TYPE_1,InventoryPolicyMinMaxSafetyStock,Max,7069.05,Min,6905.46,Safety stock,163.59,7069.05,TRUE,1,2-6-11 00:00:00,Quantity,0,day,1,(All periods),Include
1,Inventory 2,(All sites),BRAKE_PAD_TYPE_2,InventoryPolicyMinMaxSafetyStock,Max,21681.04,Min,21420.80,Safety stock,260.24,21681.04,TRUE,1,3-6-11 00:00:00,Quantity,0,day,1,(All periods),Include
2,Inventory 3,(All sites),BRAKE_PAD_TYPE_3,InventoryPolicyMinMaxSafetyStock,Max,62959.33,Min,62035.60,Safety stock,923.73,62959.33,TRUE,1,4-6-11 00:00:00,Quantity,0,day,1,(All periods),Include
3,Inventory 4,(All sites),FAN_MOTOR_TYPE_1,InventoryPolicyMinMaxSafetyStock,Max,22182.48,Min,21928.05,Safety stock,254.43,22182.48,TRUE,1,5-6-11 00:00:00,Quantity,0,day,1,(All periods),Include
4,Inventory 5,(All sites),FAN_MOTOR_TYPE_2,InventoryPolicyMinMaxSafetyStock,Max,28404.58,Min,27909.38,Safety stock,495.20,28404.58,TRUE,1,6-6-11 00:00:00,Quantity,0,day,1,(All periods),Include
5,Inventory 6,(All sites),LED_PANEL_TYPE_1,InventoryPolicyMinMaxSafetyStock,Max,38551.00,Min,38053.72,Safety stock,497.28,38551.00,TRUE,1,7-6-11 00:00:00,Quantity,0,day,1,(All periods),Include
6,Inventory 7,(All sites),LED_PANEL_TYPE_2,InventoryPolicyMinMaxSafetyStock,Max,23887.20,Min,23188.66,Safety stock,698.55,23887.21,TRUE,1,8-6-11 00:00:00,Quantity,0,day,1,(All periods),Include


##  Facility Expense Table (AnyLogistix Format)

This table defines **cost parameters** for each distribution center (DC) in your AnyLogistix simulation model. It captures three main types of facility-level expenses derived from your `forecast_df`.

---

###  Cost Components Computed

| Expense Type     | Formula / Logic                                                                 |
|------------------|----------------------------------------------------------------------------------|
| `carryingCost`   | `unit_cost × holding_cost_rate` (averaged per City)                             |
| `initialCost`    | `(365 × daily_demand ÷ reorder_point_units) × ordering_cost_per_order`          |
| `otherCost`      | stockout cost per unit (averaged per City)                                      |

---

###  Notes

- **City → DC mapping** is used via `DC_<CityName>` as Facility name.
- `initialCost` is an annualized estimate based on reorder logic and is left unitless.
- Each facility gets **3 cost entries**:
  1. Carrying cost per unit per day
  2. Initial ordering cost per year
  3. Stockout cost per day per unit

---


In [103]:
import pandas as pd

def generate_facility_expense_table(forecast_df):
    df = forecast_df.copy()

    if 'City' not in df.columns:
        df['City'] = df['Location']

    # Daily demand
    df['daily_demand'] = df['forecasted_demand']

    # Avoid division by zero in reorder point
    df['reorder_point_units'] = df['reorder_point_units'].replace(0, pd.NA)

    # Estimate initial cost using reorder point logic
    df['initial_cost'] = (
        (365 * df['daily_demand']) / df['reorder_point_units']
    ) * df['ordering_cost_per_order']
    df['initial_cost'] = df['initial_cost'].fillna(0)

    # Carrying cost per day
    df['carrying_cost_per_day'] = df['unit_cost'] * df['holding_cost_rate']

    # Grouped cost table by facility
    grouped = df.groupby('City').agg({
        'carrying_cost_per_day': 'mean',
        'initial_cost': 'mean',  # Total cost across SKUs
        'stockout_cost_per_unit': 'mean'
    }).reset_index()

    # Format records
    records = []
    for i, row in grouped.iterrows():
        facility = f"DC_{row['City']}"
        records.extend([
            {
                "ID": f"Facility Expense {i*3+1}",
                "Facility": facility,
                "Expense Type": "carryingCost",
                "Value": round(row['carrying_cost_per_day'], 3),
                "Currency": "USD",
                "Time Unit": "day",
                "Product Unit": "m³",
                "Time Period": "(All periods)"
            },
            {
                "ID": f"Facility Expense {i*3+2}",
                "Facility": facility,
                "Expense Type": "initialCost",
                "Value": round(row['initial_cost'], 2),
                "Currency": "USD",
                "Time Unit": "",
                "Product Unit": "",
                "Time Period": "(All periods)"
            },
            {
                "ID": f"Facility Expense {i*3+3}",
                "Facility": facility,
                "Expense Type": "otherCost",
                "Value": round(row['stockout_cost_per_unit'], 2),
                "Currency": "USD",
                "Time Unit": "day",
                "Product Unit": "",
                "Time Period": "(All periods)"
            }
        ])

    return pd.DataFrame(records)

# Export
facility_expense_df = generate_facility_expense_table(forecast_df)
facility_expense_df.to_csv("facility_expense_table.csv", index=False)
facility_expense_df

,ID,Facility,Expense Type,Value,Currency,Time Unit,Product Unit,Time Period
0,Facility Expense 1,"DC_Austin, TX",carryingCost,0.633,USD,day,m³,(All periods)
1,Facility Expense 2,"DC_Austin, TX",initialCost,31323.210,USD,,,(All periods)
2,Facility Expense 3,"DC_Austin, TX",otherCost,11.610,USD,day,,(All periods)
3,Facility Expense 4,"DC_Dallas, TX",carryingCost,1.035,USD,day,m³,(All periods)
4,Facility Expense 5,"DC_Dallas, TX",initialCost,29517.680,USD,,,(All periods)
5,Facility Expense 6,"DC_Dallas, TX",otherCost,16.610,USD,day,,(All periods)
6,Facility Expense 7,"DC_Green Bay, WI",carryingCost,1.115,USD,day,m³,(All periods)
7,Facility Expense 8,"DC_Green Bay, WI",initialCost,21529.280,USD,,,(All periods)
8,Facility Expense 9,"DC_Green Bay, WI",otherCost,18.210,USD,day,,(All periods)
9,Facility Expense 10,"DC_Houston, TX",carryingCost,0.933,USD,day,m³,(All periods)


#  AnyLogistix Locations Table

The **Locations Table** is an essential input file for supply chain simulation in **AnyLogistix**. It defines the geographic and structural details of all key nodes (e.g., customers, DCs, suppliers) in the network.

---

##  Purpose

To serve as the master reference for all physical locations used in the model, ensuring correct distance calculations, mapping, and network visualization.

---

##  Field Descriptions

| Column Name              | Description                                                                 |
|--------------------------|-----------------------------------------------------------------------------|
| `ID`                     | Unique identifier for the location (usually same as city name).            |
| `Code`                   | Short internal code for the location. Matches `Location` from raw data.    |
| `Name`                   | Descriptive label for display. Often same as `City`.                       |
| `City`                   | Full city name with state abbreviation (e.g., `"Dallas, TX"`).             |
| `Region`                 | Extracted state code or region from the city name (e.g., `"TX"`).          |
| `Country`                | Country where the location is situated (e.g., `"USA"`).                    |
| `Address`                | Optional field for detailed address. Often left blank.                     |
| `Latitude`               | Geographic latitude (in decimal degrees).                                  |
| `Longitude`              | Geographic longitude (in decimal degrees).                                 |
| `Autofill Coordinates`   | Set to `"FALSE"` to prevent AnyLogistix from overwriting coordinates.      |

---



### We will make the city location table and cusotmer location table then concatenate them for a export ready location table.



### City Location Table

In [146]:
# Create the Locations table for AnyLogistix
locations_table = simulation_demand_df[['Location', 'City', 'Latitude', 'Longitude']].drop_duplicates()

# Add static fields for required columns
locations_table['ID'] = locations_table['City']
locations_table['Code'] = locations_table['Location']
locations_table['Name'] = locations_table['City']
locations_table['Region'] = locations_table['City'].apply(lambda x: x.split(',')[-1].strip() if ',' in x else 'N/A')
locations_table['Country'] = 'USA'
locations_table['Address'] = ''
locations_table['Autofill Coordinates'] = 'FALSE'

# Reorder columns to match AnyLogistix format
locations_table = locations_table[[
    'ID', 'Code', 'Name', 'City', 'Region', 'Country',
    'Address', 'Latitude', 'Longitude', 'Autofill Coordinates'
]]

# Optional: Save to CSV
locations_table.to_csv("locations_table_anylogistix.csv", index=False)

print(" AnyLogistix Locations Table Preview:")
locations_table


 AnyLogistix Locations Table Preview:


,ID,Code,Name,City,Region,Country,Address,Latitude,Longitude,Autofill Coordinates
0,"Austin, TX",TX_3,"Austin, TX","Austin, TX",TX,USA,,30.2711,-97.7437,FALSE
2,"San Diego, CA",CA_2,"San Diego, CA","San Diego, CA",CA,USA,,32.7174,-117.1628,FALSE
3,"Los Angeles, CA",CA_1,"Los Angeles, CA","Los Angeles, CA",CA,USA,,34.0537,-118.2428,FALSE
4,"Madison, WI",WI_2,"Madison, WI","Madison, WI",WI,USA,,43.0748,-89.3838,FALSE
6,"Green Bay, WI",WI_3,"Green Bay, WI","Green Bay, WI",WI,USA,,44.5126,-88.0126,FALSE
7,"Sacramento, CA",CA_4,"Sacramento, CA","Sacramento, CA",CA,USA,,38.5811,-121.4939,FALSE
9,"Dallas, TX",TX_1,"Dallas, TX","Dallas, TX",TX,USA,,32.7763,-96.7969,FALSE
12,"Milwaukee, WI",WI_1,"Milwaukee, WI","Milwaukee, WI",WI,USA,,43.0386,-87.9091,FALSE
14,"Houston, TX",TX_2,"Houston, TX","Houston, TX",TX,USA,,29.7589,-95.3677,FALSE
20,"San Jose, CA",CA_3,"San Jose, CA","San Jose, CA",CA,USA,,37.3362,-121.8906,FALSE


### Customer Locations

In [142]:
import random
import pandas as pd
import re
import math

# 1. Define the anchor cities with their base coordinates
anchor_cities = [
    {"City": "Dallas, TX", "Lat": 32.7763, "Lon": -96.7969},
    {"City": "San Diego, CA", "Lat": 32.7174, "Lon": -117.1628},
    {"City": "Milwaukee, WI", "Lat": 43.0386, "Lon": -87.9091},
    {"City": "San Jose, CA", "Lat": 37.3362, "Lon": -121.8906},
    {"City": "Austin, TX", "Lat": 30.2711, "Lon": -97.7437},
    {"City": "Green Bay, WI", "Lat": 44.5126, "Lon": -88.0126},
    {"City": "Madison, WI", "Lat": 43.0748, "Lon": -89.3838},
    {"City": "Sacramento, CA", "Lat": 38.5811, "Lon": -121.4939},
    {"City": "Houston, TX", "Lat": 29.7589, "Lon": -95.3677},
    {"City": "Los Angeles, CA", "Lat": 34.0537, "Lon": -118.2428}
]

city_map = {c["City"]: c for c in anchor_cities}

# 2. The input sequence
location_sequence = """Dallas, TX Dallas, TX Dallas, TX Green Bay, WI Madison, WI Dallas, TX San Jose, CA 
Sacramento, CA Milwaukee, WI San Jose, CA Austin, TX Dallas, TX Madison, WI Dallas, TX 
Sacramento, CA Los Angeles, CA Milwaukee, WI San Diego, CA Dallas, TX Milwaukee, WI 
Dallas, TX Dallas, TX Los Angeles, CA Dallas, TX Sacramento, CA Dallas, TX Dallas, TX 
Madison, WI Madison, WI Sacramento, CA San Jose, CA Dallas, TX Houston, TX Los Angeles, CA 
Houston, TX Dallas, TX Green Bay, WI Milwaukee, WI Dallas, TX Los Angeles, CA Dallas, TX 
Dallas, TX San Jose, CA Sacramento, CA San Diego, CA Sacramento, CA San Jose, CA 
Dallas, TX Los Angeles, CA Green Bay, WI Dallas, TX Milwaukee, WI Dallas, TX Austin, TX 
San Jose, CA San Diego, CA Austin, TX Austin, TX Austin, TX Dallas, TX Madison, WI 
Houston, TX Green Bay, WI San Jose, CA Dallas, TX Los Angeles, CA San Jose, CA Austin, TX 
Los Angeles, CA Los Angeles, CA Los Angeles, CA Dallas, TX Dallas, TX Los Angeles, CA 
Sacramento, CA Austin, TX San Diego, CA Austin, TX San Jose, CA San Jose, CA Austin, TX 
San Jose, CA Austin, TX San Jose, CA Dallas, TX Los Angeles, CA Austin, TX Dallas, TX 
San Jose, CA Los Angeles, CA Austin, TX Austin, TX Houston, TX Green Bay, WI San Diego, CA 
Houston, TX San Jose, CA San Diego, CA San Diego, CA San Diego, CA"""

pattern = r'(Dallas, TX|San Diego, CA|Milwaukee, WI|San Jose, CA|Austin, TX|Green Bay, WI|Madison, WI|Sacramento, CA|Houston, TX|Los Angeles, CA)'
locations = re.findall(pattern, location_sequence)

random.seed(42)
simulated_data = []

# 250 km = ~2.25° lat, scale lon with cos(lat)
MAX_LAT_OFFSET = 2.25  # degrees

for idx, city_name in enumerate(locations):
    anchor = city_map.get(city_name)
    if anchor:
        base_lat = anchor["Lat"]
        base_lon = anchor["Lon"]

        # Longitude offset adjusted for latitude
        lat_offset = random.uniform(-MAX_LAT_OFFSET, MAX_LAT_OFFSET)
        lon_offset = random.uniform(-MAX_LAT_OFFSET, MAX_LAT_OFFSET) / math.cos(math.radians(base_lat))

        new_lat = base_lat + lat_offset
        new_lon = base_lon + lon_offset

        simulated_data.append({
            "ID": f"Customer {idx + 1}",
            "Code": city_name,
            "Latitude": round(new_lat, 6),
            "Longitude": round(new_lon, 6)
        })

df = pd.DataFrame(simulated_data)


df.to_csv("simulated_customer_locations.csv", index=False)
df=df.merge(anylogix_customers_table,on='ID',how="inner")
df

,ID,Code,Latitude,Longitude,Name,Type,Location,Inclusion Type,Icon
0,Customer 1,"Dallas, TX",33.403721,-99.339093,SuperCustomer_41,Customer,"Green Bay, WI",Include,116
1,Customer 2,"Dallas, TX",31.763932,-98.278305,SuperCustomer_92,Customer,"Dallas, TX",Include,116
2,Customer 3,"Dallas, TX",33.840420,-95.851186,SuperCustomer_84,Customer,"Austin, TX",Include,116
3,Customer 4,"Green Bay, WI",46.277408,-90.619226,SuperCustomer_32,Customer,"Dallas, TX",Include,116
4,Customer 5,"Madison, WI",42.723448,-92.280474,SuperCustomer_74,Customer,"Dallas, TX",Include,116
...,...,...,...,...,...,...,...,...,...
95,Customer 96,"Houston, TX",29.407367,-96.515634,SuperCustomer_50,Customer,"San Jose, CA",Include,116
96,Customer 97,"San Jose, CA",36.210329,-119.495030,SuperCustomer_45,Customer,"San Diego, CA",Include,116
97,Customer 98,"San Diego, CA",32.461488,-115.230100,SuperCustomer_67,Customer,"Austin, TX",Include,116
98,Customer 99,"San Diego, CA",32.943864,-119.566509,SuperCustomer_69,Customer,"Houston, TX",Include,116


In [147]:
# Merge with customer metadata
df = df.merge(anylogix_customers_table, on='ID', how='inner')

# Format Name and ID to include "Location" suffix
df['ID'] = df['ID'] + ' Location'
df['Name'] = df['ID']  # Same as updated ID

# Extract city and assign region/country/address
df['City'] = df['Code']
df['Region'] = df['City'].str.extract(r',\s*(\w{2})')[0]  # e.g., 'TX' from 'Dallas, TX'
df['Region'] = df['Region'].replace({
    'TX': 'Texas',
    'CA': 'California',
    'WI': 'Wisconsin'
})
df['Country'] = 'United States'
df['Address'] = ''
df['Autofill Coordinates'] = 'FALSE'

# Final column order
df_final = df[[
    'ID', 'Code', 'Name', 'City', 'Region', 'Country', 'Address',
    'Latitude', 'Longitude', 'Autofill Coordinates'
]]

# Save to CSV
df_final.to_csv("formatted_customer_locations.csv", index=False)
df_final

MergeError: Passing 'suffixes' which cause duplicate columns {'Name_x'} is not allowed.

## Final Location Table for Anylogistix

In [148]:
final=pd.concat([df_final,locations_table])
final

,ID,Code,Name,City,Region,Country,Address,Latitude,Longitude,Autofill Coordinates
0,Customer 1 Location,"Dallas, TX",Customer 1 Location,"Dallas, TX",Texas,United States,,33.403721,-99.339093,FALSE
1,Customer 2 Location,"Dallas, TX",Customer 2 Location,"Dallas, TX",Texas,United States,,31.763932,-98.278305,FALSE
2,Customer 3 Location,"Dallas, TX",Customer 3 Location,"Dallas, TX",Texas,United States,,33.840420,-95.851186,FALSE
3,Customer 4 Location,"Green Bay, WI",Customer 4 Location,"Green Bay, WI",Wisconsin,United States,,46.277408,-90.619226,FALSE
4,Customer 5 Location,"Madison, WI",Customer 5 Location,"Madison, WI",Wisconsin,United States,,42.723448,-92.280474,FALSE
...,...,...,...,...,...,...,...,...,...,...
7,"Sacramento, CA",CA_4,"Sacramento, CA","Sacramento, CA",CA,USA,,38.581100,-121.493900,FALSE
9,"Dallas, TX",TX_1,"Dallas, TX","Dallas, TX",TX,USA,,32.776300,-96.796900,FALSE
12,"Milwaukee, WI",WI_1,"Milwaukee, WI","Milwaukee, WI",WI,USA,,43.038600,-87.909100,FALSE
14,"Houston, TX",TX_2,"Houston, TX","Houston, TX",TX,USA,,29.758900,-95.367700,FALSE


##  Products Table for AnyLogistix

This **Products Table** defines the catalog of items to be used across your supply chain simulation. Each product includes essential cost and pricing data for financial analysis within AnyLogistix.

---

###  Table Details

| Column          | Description                                                        |
|-----------------|--------------------------------------------------------------------|
| `ID`            | Product type identifier (from `part_type`)                         |
| `Name`          | Same as ID; used for display purposes                              |
| `Unit`          | Product unit of measurement — set as `pcs`                         |
| `Selling Price` | Mean per-unit revenue (based on cleaned forecasted demand)         |
| `Unit Cost`     | Mean per-unit procurement or production cost                       |
| `Currency`      | Currency used in simulation — set as `USD`                         |

---

###  Cleaning Rules Applied

- Excluded rows where `forecasted_demand` ≤ 1 or missing — ensures realistic per-unit values.
- Aggregated `unit_cost` and `revenue` (renamed to `selling_price`) at `part_type` level.
- Rounded monetary values to two decimal places.

---




In [140]:
import pandas as pd

# Step 1: Filter required columns
required_cols = ['part_id', 'part_type', 'unit_cost', 'revenue', 'annual_demand_units']
missing = [col for col in required_cols if col not in forecast_df.columns]
assert not missing, f"Missing columns: {missing}"

# Step 2: Clean zero or missing demand and calculate selling price
forecast_df = forecast_df.copy()

# Remove rows with zero or missing forecasted_demand to avoid division errors
forecast_df = forecast_df[forecast_df['forecasted_demand'].notna() & (forecast_df['forecasted_demand'] > 1)]

# Calculate per-unit selling price
forecast_df['selling_price'] = forecast_df['revenue']

# Step 3: Aggregate by product type
products_df = forecast_df.groupby('part_type').agg({
    'unit_cost': 'mean',
    'selling_price': 'mean',
    
}).reset_index()

# Step 4: Final formatting
products_df.rename(columns={
    'part_type': 'ID',
    'unit_cost': 'Unit Cost',
    'selling_price': 'Selling Price',
}, inplace=True)

products_df['Name'] = products_df['ID']
products_df['Unit'] = 'pcs'
products_df['Currency'] = 'USD'

# Reorder and round
products_df = products_df[['ID', 'Name', 'Unit', 'Selling Price', 'Unit Cost', 'Currency']]
products_df['Selling Price'] = products_df['Selling Price'].round(2)
products_df['Unit Cost'] = products_df['Unit Cost'].round(2)


# Sort by revenue descending
products_df = products_df.sort_values(by='Name', ascending=True).reset_index(drop=True)

# Save to CSV
products_df.to_csv("products_table_cleaned.csv", index=False)

# Final preview
products_df


,ID,Name,Unit,Selling Price,Unit Cost,Currency
0,BRAKE_PAD_TYPE_1,BRAKE_PAD_TYPE_1,pcs,5.74,2.40,USD
1,BRAKE_PAD_TYPE_2,BRAKE_PAD_TYPE_2,pcs,11.19,4.14,USD
2,BRAKE_PAD_TYPE_3,BRAKE_PAD_TYPE_3,pcs,7.07,2.48,USD
3,FAN_MOTOR_TYPE_1,FAN_MOTOR_TYPE_1,pcs,8.88,4.56,USD
4,FAN_MOTOR_TYPE_2,FAN_MOTOR_TYPE_2,pcs,5.30,3.25,USD
5,LED_PANEL_TYPE_1,LED_PANEL_TYPE_1,pcs,7.79,4.84,USD
6,LED_PANEL_TYPE_2,LED_PANEL_TYPE_2,pcs,2.96,2.02,USD


In [ ]:
simulation_demand_df['Product'].value_counts()

Product
BRAKE_PAD_TYPE_3    632
LED_PANEL_TYPE_1    345
BRAKE_PAD_TYPE_2    267
FAN_MOTOR_TYPE_2    252
FAN_MOTOR_TYPE_1    222
LED_PANEL_TYPE_2    190
BRAKE_PAD_TYPE_1     92
Name: count, dtype: int64

#  Completing the AnyLogistix Simulation: Final Table Creation Strategy

After programmatically generating core supply chain tables like **Locations**, **Facility Expenses**, **Products**, **Inventory Policies**, and **Customer Coordinates**, the following approach outlines how the **remaining required tables** will be constructed to finalize the model for simulation in **AnyLogistix**.

---

##  Tables Already Created via Python Scripts

| Table Name           | Status  | Description |
|----------------------|---------|-------------|
| `Locations`          | ✅ Done | Geographic coordinates for all demand and supply nodes. |
| `Facility Expenses`  | ✅ Done | Captures carrying, initial, and stockout costs per facility. |
| `Products`           | ✅ Done | Cleaned product catalog with cost and selling price details. |
| `Inventory`          | ✅ Done | Min-Max-Safety stock policy table by product and facility. |
| `Customers`          | ✅ Done | Simulated customer list with coordinates and IDs. |
| `BOM`                          |✅ Done      | Define bill of materials for each finished product. |
| `Distribution Centers`         | ✅ Done     | Site types, capacities, and coordinates. |
| `Demand`                       | ✅ Done    | Historical/forecasted demand per customer-product-period. |
---

##  Remaining Tables and Construction Approach

All remaining tables will be created using a **hybrid method**:
- **Excel**: For structured tabular data input with formulas, dropdowns, and validation.
- **AnyLogistix GFA & NO Modules**: For configuring process-specific tables like factories, paths, production settings, and transportation logic.

| Table Name                     | Method     | Notes |
|--------------------------------|------------|-------|
| `Scenario Settings`            | Excel      | Define scenario name, units, and global parameters. |
| `Factories`                    | Excel      | Define manufacturing sites, capacities, and setup costs. |
| `Groups`, `Groups Customers`, `Groups Sites`, `Groups Suppliers` | Excel | For segmentation or zone-wise management. |
| `Unit Conversions`             | Excel      | If non-standard units (e.g., pallets, kg) are used. |
| `Paths`                        | ALX GFA    | Define allowed paths between sites. |
| `Periods`                      | Excel      | Define discrete simulation periods (e.g., days or weeks). |
| `Processing Cost`              | Excel      | Cost for value-adding activities like assembly, packaging. |
| `Production`                   | Excel      | Output capacity, cycle times, lot sizes. |
| `Shipping`                     | Excel      | Define shipping policies like lot size, shipment frequency. |
| `Shipping Destinations`        | Excel      | Explicit destinations and mapping for each DC/factory. |
| `Sourcing`                     | Excel      | Which facility sources which product from whom. |
| `Sourcing Sources`            | Excel      | Product-level mapping for inbound sourcing. |
| `Suppliers`                    | Excel      | Raw material or component suppliers and lead times. |
| `Tariffs`                      | Excel      | Cross-border or zone-wise tariffs. |
| `Vehicle Types`                | Excel      | Transport modes: truck, rail, air, etc. |
| `Experiments`, `Experiment Statistics Settings`, `Experiment Dashboards`, `Experiment Metrics` | ALX Interface | To configure and track simulation KPIs. |
| `Project Units`, `Project Unit Conversions` | Excel | Set currency, distance, volume, weight units for the model. |
| `Icons`                        | ALX Interface | For visual layer customization. |

---

##  Workflow Summary

1.  **Data-Driven Tables**: Generated using Python scripts (Locations, Products, Customers, Inventory, Expenses).
2.  **Excel-Driven Tables**: Structured manually based on data from Python exports and simulation planning logic.
3.  **AnyLogistix GUI (GFA & NO)**: Used to model paths, define experiments, set constraints, and run simulations.
4.  **Final Simulation**: After all tables are populated and imported, the model will be tested and validated using **AnyLogistix Experiment Modules** (Performance, Optimization, Risk Analysis, etc.).

---

## Final Deliverables

- `*.csv` files for all input tables
- `.alx` project file with simulation logic
- Documented assumptions, cost structures, and policies
- Dashboard exports and metrics after experiment runs

---

By combining **programmatic automation**, **structured Excel planning**, and **AnyLogistix's GFA + NO interface**, this approach ensures a scalable and simulation-ready supply chain model for robust decision-making.
